El objetivo es programar una función que reciba como input un texto de usuario y devuelva los fragmentos de texto (chunks) que hagan referencia a las comidas y cantidades que ha solicitado. No es necesario, ni es el objetivo de este ejercicio, construir un clasificador de intención previo a esta función, sino simplemente una función que presuponemos recibe una frase con la intención 'Pedir_comida'. Tampoco es objetivo normalizar la salida (por ej.: no es necesario convertir 'tres' a '3' ni 'pizzas' a 'pizza'). Es, por tanto, un ejercicio de mínimos.

el alumno deberá usar un NaiveBayesClassifier, en lugar del MaxEntClassifier, para localizar los elementos descritos anteriormente (comida y cantidad)

Se deberá comenzar la práctica por el nivel más básico de dificultad (RegexParser) y, en caso de conseguirlo, añadir los siguientes niveles de forma sucesiva. De esta forma, el entregable contendrá todas y cada una de las tres formas de solucionar el problema. No basta, por tanto, con incluir, por ejemplo, únicamente un NaiveBayesClassifier, hay que incluir también las otras dos formas si se quiere obtener la máxima puntuación.

Para llevar a cabo la práctica, deberá construirse una cadena NLP con NLTK, con los siguientes elementos:

    segmentación de frases,
    tokenización,
    POS tagger (analizador mofológico para el español).

A continuación, los POS tags obtenidos serán usados por el RegexParser, el UnigramParser, el BigramParser y el NaiveBayesClassifier.


In [ ]:
# Importamos las librerias necesarias
from nltk import Tree
from nltk.chunk import *
import datetime
import nltk
from nltk.corpus import cess_esp
from nltk import UnigramTagger, BigramTagger, TrigramTagger, DefaultTagger
from nltk.tag.hmm import HiddenMarkovModelTagger

In [1]:
# Creamos un corpus de ejemplo para el reconocimiento de peticiones de comida:
corpus_comida = ["Me gustaria tomar unas rabas",
                 "Quisiera un pincho de tortilla", 
                 "Me encantaria unas gambas a la plancha",
                 "Me pones un bocadillo de bacon con queso?",
                 "Podrias ponerme una ensalada de tomate?",
                 "Un bocata de lomo",
                 "Unos pimientos del piquillo",
                 "Me puedes traer un plato combinado?",
                 "Podria ser un filete poco hecho?"]

In [2]:
# Entrenamos el tagger

# Procedemos al entrenamiento utilizando los tags y corpus de cess_esp
sents = cess_esp.tagged_sents()
#Metemos en el conjunto de entrenamiento el 90% de las frases, y el restante 10% en el conjunto de test
training = []
test = []
print("Generando sentencias de aprendizaje...", datetime.datetime.now())
for i in range(len(sents)):
    if i % 10:
        training.append(sents[i])
    else:
        test.append(sents[i])

# Establecemos la cadena de taggeo, utilizando el HMM en primer lugar y haciendo que,
# si no reconoce una palabra la intente taggear usando otro tagger y asi sucesivamente.
print("Entrenando al Tagger ...", datetime.datetime.now())
default_tagger = DefaultTagger ('DEFAULT')
unigram_tagger = UnigramTagger(training, backoff=default_tagger)
bigram_tagger = BigramTagger(training, backoff=unigram_tagger)
print("Tagger entrenado...", datetime.datetime.now())

Generando sentencias de aprendizaje... 2018-08-31 23:27:42.891839
Entrenando al Tagger ... 2018-08-31 23:28:12.079350
Tagger entrenado... 2018-08-31 23:28:17.029585


In [307]:
# Una vez entrenado procedemos a analizar las frases de nuestro corpus mediante un 
# Regexp parser para obtener las entidades que aparezcan en los textos
# Definimos la gramatica que nos permita reconocer comidas y cantidades
# Definimos un pos-tagger con RegExp para calificar cantidades, comidas e ingredientes
patterns = [
    (r'([0-9]+|unas|una|un|unos)','CANT'),
    (r'bocadillo|bocata|pizza|piza|pincho|racion|plato', 'TipoCOMIDA'),
    (r'jamon|chorizo|queso|calamares|pimientos|patatas|tortilla','INGR')
]
regexp_tagger = nltk.RegexpTagger(patterns, backoff=bigram_tagger)
grammar = r"""
  Comida: {(<CANT> <TipoCOMIDA | INGR>)} # Comida
  ComidaElaborada: {<Comida> (<INGR>+|(<INGR>*<.*><INGR>)+)}
  ComidaIndefinida: {<TipoCOMIDA>| <INGR>}
  """
regex_parser = nltk.RegexpParser(grammar)
print(regex_parser)

chunk.RegexpParser with 3 stages:
RegexpChunkParser with 1 rules:
    Comida   <ChunkRule: '(<CANT> <TipoCOMIDA | INGR>)'>
RegexpChunkParser with 1 rules:
       <ChunkRule: '<Comida> (<INGR>+|(<INGR>*<.*><INGR>)+)'>
RegexpChunkParser with 1 rules:
       <ChunkRule: '<TipoCOMIDA>| <INGR>'>


In [356]:
# Funcion de extraccion de informacion de pedidos. Recibe como parametro un texto con
# la solicitud del cliente y devuelve el pedido (alimento y cantidad), si es que lo hay
def extract_pedidos (solicitud):
    # Tokenizamos la frase
    tokens = nltk.word_tokenize(solicitud)
    taggeado = regexp_tagger.tag(tokens)
    # print(taggeado)
    arbol = regex_parser.parse(taggeado) # (taggeado, trace = True)
    # print(arbol)
    # Convertimos en estructura de arbol, que mantiene relacion con su predecesor:
    parentTree = nltk.tree.ParentedTree.convert(arbol)
    # Recorremos el arbol buscando el contexto de cada tag, para quedarnos con los
    # pedidos completos que haya dentro de cada solicitud
    comidas = []
    for subarbol in parentTree.subtrees(filter=lambda t: t.label()=='ComidaElaborada'
                                       or t.label()=='Comida' or t.label()=='ComidaIndefinida'):
        print('Subarbol --> ', subarbol.label(), ',    Pertenece a: ', subarbol.parent().label())    
        print (subarbol)
        # Si el padre del subarbol sintactico es S, examinamos el subarbol 
        # ya que contiene un pedido. Si el padre no es S, es la raiz Comida de ComidaElaborada
        pedidoParcial = {}
        if subarbol.parent().label() == 'S':
            # Vamos rellenando los distintos elementos del pedido
            ingredientes = []
            for hoja in subarbol.leaves():
                if hoja[1] == 'CANT':
                    pedidoParcial['cantidad'] = hoja[0]
                elif hoja[1] == 'INGR':
                    ingredientes.append(hoja[0])
                elif hoja[1] == 'TipoCOMIDA':
                    pedidoParcial['tipoComida'] = hoja[0]
            pedidoParcial['ingredientes'] = ingredientes
        comidas.append(pedidoParcial)
        print(pedidoParcial)
    print(comidas)

In [357]:
# Prueba 1
sentencia1 = "Quiero un jamon y un plato de calamares, pimientos y patatas"
extract_pedidos(sentencia1)

Subarbol -->  Comida ,    Pertenece a:  S
(Comida un/CANT jamon/INGR)
Subarbol -->  Comida ---> Hoja : ('un', 'CANT') un
Cantidad =  un
Subarbol -->  Comida ---> Hoja : ('jamon', 'INGR') jamon
{'cantidad': 'un', 'ingredientes': ['jamon']}
Subarbol -->  ComidaElaborada ,    Pertenece a:  S
(ComidaElaborada
  (Comida un/CANT plato/TipoCOMIDA)
  de/sps00
  calamares/INGR
  ,/Fc
  pimientos/INGR
  y/cc
  patatas/INGR)
Subarbol -->  ComidaElaborada ---> Hoja : ('un', 'CANT') un
Cantidad =  un
Subarbol -->  ComidaElaborada ---> Hoja : ('plato', 'TipoCOMIDA') plato
Subarbol -->  ComidaElaborada ---> Hoja : ('de', 'sps00') de
Subarbol -->  ComidaElaborada ---> Hoja : ('calamares', 'INGR') calamares
Subarbol -->  ComidaElaborada ---> Hoja : (',', 'Fc') ,
Subarbol -->  ComidaElaborada ---> Hoja : ('pimientos', 'INGR') pimientos
Subarbol -->  ComidaElaborada ---> Hoja : ('y', 'cc') y
Subarbol -->  ComidaElaborada ---> Hoja : ('patatas', 'INGR') patatas
{'cantidad': 'un', 'tipoComida': 'plato', 'i

In [319]:
# Prueba 2
sentencia2 = "Quiero un jamon y una piza de chorizo, queso y 5 bocadillo de tortilla"
extract_pedidos(sentencia2)

# Input:
 <vmip1s0>  <CANT>  <INGR>  <cc>  <CANT>  <TipoCOMIDA>  <sps00>  <INGR>  <Fc>  <INGR>  <cc>  <CANT>  <TipoCOMIDA>  <sps00>  <INGR> 
# Comida:
 <vmip1s0> {<CANT>  <INGR>} <cc> {<CANT>  <TipoCOMIDA>} <sps00>  <INGR>  <Fc>  <INGR>  <cc> {<CANT>  <TipoCOMIDA>} <sps00>  <INGR> 
# Input:
 <vmip1s0>  <Comida>  <cc>  <Comida>  <sps00>  <INGR>  <Fc>  <INGR>  <cc>  <Comida>  <sps00>  <INGR> 
# :
 <vmip1s0>  <Comida>  <cc> {<Comida>  <sps00>  <INGR>  <Fc>  <INGR>} <cc> {<Comida>  <sps00>  <INGR>}
# Input:
 <vmip1s0>  <Comida>  <cc>  <ComidaElaborada>  <cc>  <ComidaElaborada> 
# :
 <vmip1s0>  <Comida>  <cc>  <ComidaElaborada>  <cc>  <ComidaElaborada> 
(S
  Quiero/vmip1s0
  (Comida un/CANT jamon/INGR)
  y/cc
  (ComidaElaborada
    (Comida una/CANT piza/TipoCOMIDA)
    de/sps00
    chorizo/INGR
    ,/Fc
    queso/INGR)
  y/cc
  (ComidaElaborada
    (Comida 5/CANT bocadillo/TipoCOMIDA)
    de/sps00
    tortilla/INGR))
Subarbol -->  Comida ,    Pertenece a:  S
(Comida un/CANT jamon/INGR)
Suba

In [314]:
# Prueba 3
sentencia3 = "Quiero un jamon y pizza"
extract_pedidos(sentencia3)

# Input:
 <vmip1s0>  <CANT>  <INGR>  <cc>  <TipoCOMIDA> 
# Comida:
 <vmip1s0> {<CANT>  <INGR>} <cc>  <TipoCOMIDA> 
# Input:
 <vmip1s0>  <Comida>  <cc>  <TipoCOMIDA> 
# :
 <vmip1s0>  <Comida>  <cc>  <TipoCOMIDA> 
# Input:
 <vmip1s0>  <Comida>  <cc>  <TipoCOMIDA> 
# :
 <vmip1s0>  <Comida>  <cc> {<TipoCOMIDA>}
(S
  Quiero/vmip1s0
  (Comida un/CANT jamon/INGR)
  y/cc
  (ComidaIndefinida pizza/TipoCOMIDA))
Subarbol -->  Comida ,    Pertenece a:  S
(Comida un/CANT jamon/INGR)
Subarbol -->  Comida ---> Hoja : ('un', 'CANT')
Subarbol -->  Comida ---> Hoja : ('jamon', 'INGR')
Subarbol -->  ComidaIndefinida ,    Pertenece a:  S
(ComidaIndefinida pizza/TipoCOMIDA)
Subarbol -->  ComidaIndefinida ---> Hoja : ('pizza', 'TipoCOMIDA')
